# PaddleHub一键视频动漫化
## 你曾想过你把自己拍的一段街景视频变成日系动漫风格吗？现在就给你马上安排！！！

### 原图
![](https://ai-studio-static-online.cdn.bcebos.com/7deee9f92de84b33ba2ca44adb79fa2974406385ae7a4105bcb1c219207c4e3f)
### 风格化后的图
![](https://ai-studio-static-online.cdn.bcebos.com/24b23831735042f089aac5d7be84744f18c1488b92eb4317a52a06c00bf359f5)

作者技术有限，不会弄GIF，那就拿其中一帧来展示(┳_┳)
### 如何秒变一波视频呢？
### 咳咳咳。。。通过对视频进行拆分成一张张图片，将每一张图片都风格化，再将图片合成为视频，添加原有的音频就Okay了哦！
### 有哪些风格呢？
### 请移步到**3、选择视频及模板**
### 话不多说，直接上程序


**Tips：风格化照片时会比较久，敬请大家留意 & 体验前 建议跳转到10去清理临时文件**

# 0、BUG
在生成新版本时候，空文件夹无法加入，即使加入成功后，别人fork也无法显示。望修复，谢谢:)

In [2]:
!mkdir -p work/mp4_img work/mp4_img3 work/output

# 1、安装PaddleHub

In [36]:
!pip install paddlehub -U -i https://pypi.tuna.tsinghua.edu.cn/simple #用了清华的镜像源

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Requirement already up-to-date: paddlehub in /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages (1.8.3)


# 2、设置GPU环境

In [37]:
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline

env: CUDA_VISIBLE_DEVICES=0


# 3、导入相应的库

In [38]:
import cv2
import paddlehub as hub
import os

# 4、选择视频及模板

**Tips：可在此处更改风格哦**

这里可以更换很多风格，想了解更多风格，[请点击此处](https://www.paddlepaddle.org.cn/hublist?filter=en_category&value=GANs)

In [39]:
input_video = 'test.mp4'
model = hub.Module('animegan_v2_shinkai_33', use_gpu=True) #这里用的是animegan_v2_shinkai_33(新海诚动漫风格)

[2021-01-30 22:39:36,459] [    INFO] - Installing animegan_v2_shinkai_33 module
[2021-01-30 22:39:36,462] [    INFO] - Module animegan_v2_shinkai_33 already installed in /home/aistudio/.paddlehub/modules/animegan_v2_shinkai_33


# 5、将视频转化为图片

Tips：可以用`ls work/mp4_img | wc -w`命令到终端看一下完成的图片数量

In [40]:
def transform_video_to_image(video_file_path, img_path):
    '''
    将视频中每一帧保存成图片
    '''
    video_capture = cv2.VideoCapture(video_file_path)
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    count = 0
    while(True):
        ret, frame = video_capture.read() 
        if ret:
            cv2.imwrite(img_path + '%d.jpg' % count, frame)
            count += 1
        else:
            break
    video_capture.release()
    print('视频图片保存成功, 共有 %d 张' % count)
    return fps,count

In [41]:
# 将视频中每一帧保存成图片
fps,count = transform_video_to_image(input_video, 'work/mp4_img/')

视频图片保存成功, 共有 431 张


# 6、将图片转换风格

*备注：运行时间可能会很久哦*

Tips：可以用`ls work/mp4_img3 | wc -w`命令到终端看一下完成的图片数量

In [42]:
def get_combine_img(input_file_patha):
    #Pathname=""
    output_file_path="work/mp4_img2/"
    input_file_path="work/mp4_img/"+input_file_patha
    #print(input_file_path)
    #print(output_file_path)
    result = model.style_transfer(images=[cv2.imread(input_file_path)],visualization=True,output_dir=output_file_path)
    for root, dirs, files in os.walk(output_file_path):
        fils=files
    files=''.join(files)
    #print(files)
    dict1="mv "+output_file_path+files+" "+output_file_path+input_file_patha
    os.system(dict1)
    dict1="cp "+output_file_path+input_file_patha+" "+"./work/mp4_img3/"+input_file_patha
    #print(dict1)
    os.system(dict1)
    os.system("rm -rf ./work/mp4_img2")

In [43]:
def transform():
    os.system("mkdir ./work/mp4_img3")
    for i in range(0,count):
        name=str(i)+".jpg"
        #print(name)
        get_combine_img(name)
    print('视频图片转换成功, 共有 %d 张' % (i+1))
transform()

视频图片转换成功, 共有 431 张


# 7、将图片合成为视频

In [44]:
def combine_image_to_video(comb_path, output_file_path, fps, is_print=False):
    '''
        合并图像到视频
    '''
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')    
    
    file_items = os.listdir(comb_path)
    file_len = len(file_items)
    # print(comb_path, file_items)
    if file_len > 0 :
        temp_img = cv2.imread(os.path.join(comb_path, file_items[0]))
        img_height, img_width = temp_img.shape[0], temp_img.shape[1]
        
        out = cv2.VideoWriter(output_file_path, fourcc, fps, (img_width, img_height))

        for i in range(file_len):
            pic_name = os.path.join(comb_path, str(i)+".jpg")
            if is_print:
                print(i+1,'/', file_len, ' ', pic_name)
            img = cv2.imread(pic_name)
            out.write(img)
        out.release()

In [45]:
import time
final_name="work/output/"+time.strftime("%Y%m%d%H%M%S", time.localtime())+".mp4"
tran_name="! ffmpeg -i work/mp4_analysis.mp4 -i work/video.mp3 -c copy "+final_name

In [46]:
combine_image_to_video('work/mp4_img3/', 'work/mp4_analysis.mp4' ,fps)

# 8、添加原有音频

In [47]:
! ffmpeg -i test.mp4 -vn work/video.mp3
os.system(tran_name)
#! ffmpeg -i work/mp4_analysis.mp4 -i work/video.mp3 -c copy output/mp4_analysis_result.mp4

ffmpeg version 2.8.15-0ubuntu0.16.04.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.10) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --

256

# 10、清除临时数据

In [3]:
! rm -rf ./work/mp4_img/*
! rm -rf ./work/mp4_img3/*
! rm -rf ./work/video.mp3
! rm -rf ./work/mp4_analysis.mp4

# 关于作者
😃姓名：曾焯淇😃

😃学历：高中😃

😃From：广东 佛山（欢迎面基）😃

我在AI Studio上获得黄金等级，点亮3个徽章，来互关呀~ [https://aistudio.baidu.com/aistudio/personalcenter/thirdview/233221](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/233221)